In [ ]:
import os

from ase.io import read as ase_read
from ase.io import write as ase_write
from ase.optimize import BFGS
from ase import Atoms


In [ ]:
hfo2_222 = ase_read("HfO2_222_MACE100725_DATA", format="lammps-data")

In [ ]:
#model_path = "../../models/HfOx_128x0e_100725_1007_from-scratch_stagetwo.model"
#traj_file = "hfo2_222_128x0e_100725_1500K.traj"
#log_file = "hfo2_222_128x0e_100725_1500K.csv"

model_path = "../../models/HfOx_128x0e_pair_repulsion_stagetwo.model"
traj_file = "hfo2_222_128x0e_pair-repulsion_1500K.traj"
log_file = "hfo2_222_128x0e_pair-repulsion_1500K.csv"



In [ ]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=model_path, device="cpu")

In [ ]:
hfo2_222.set_calculator(mace_calc)

In [ ]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
temp = 1500
sys = hfo2_222
MaxwellBoltzmannDistribution(sys,temperature_K=temp)

In [ ]:
from ase import units
from ase.md.nose_hoover_chain import NoseHooverChainNVT
dyn = NoseHooverChainNVT(sys, 1.0 * units.fs, temperature_K=temp,
                         tdamp = 50.0)

In [ ]:
from ase.io.trajectory import Trajectory

traj = Trajectory(traj_file, 'w', sys)
dyn.attach(traj.write, interval=50)

import csv
# Prepare CSV header
with open(log_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Step", "E_pot (eV)", "E_kin (eV)", "T (K)", "E_tot (eV)"])

# Logging function
def log_status():
    epot = sys.get_potential_energy()
    ekin = sys.get_kinetic_energy()
    temp = ekin / (1.5 * units.kB * len(sys))
    etot = epot + ekin
    step = dyn.nsteps
    print(f"Step {step:6d} | T = {temp:8.1f} K  | "
          f"E_pot = {epot:9.3f} eV  |  E_tot = {etot:9.3f} eV")
    with open(log_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([step, epot, ekin, temp, etot])

dyn.attach(log_status, interval=10)

In [ ]:
dyn.run(5000)